<a href="https://colab.research.google.com/github/AdamsD02/Music-Genre-Classification/blob/main/mgc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone "https://github.com/AdamsD02/Music-Genre-Classification.git"


Cloning into 'Music-Genre-Classification'...
remote: Enumerating objects: 1012, done.
remote: Total 1012 (delta 0), reused 0 (delta 0), pack-reused 1012 (from 2)
Receiving objects: 100% (1012/1012), 1.13 GiB | 30.00 MiB/s, done.
Updating files: 100% (1002/1002), done.


In [5]:
%cd "Music-Genre-Classification"

/content/Music-Genre-Classification


In [6]:
%ls

features_30_sec.csv  genres/  README.md
